# Home Credit Default Risk

https://www.kaggle.com/c/home-credit-default-risk#description

https://www.kaggle.com/codename007/home-credit-complete-eda-feature-importance/notebook

* application_{train|test}.csv
 * main table
* bureau.csv
 * 其他機構之前的 credit
* bureau_balance.csv
 * bureau.csv 的每月餘額
* POS_CASH_balance.csv
* credit_card_balance.csv
* previous_application.csv
* installments_payments.csv
* HomeCredit_columns_description.csv

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [2]:
application_train = pd.read_csv('../input/application_train.csv')
#POS_CASH_balance = pd.read_csv('../input/POS_CASH_balance.csv')
#bureau_balance = pd.read_csv('../input/bureau_balance.csv')
#previous_application = pd.read_csv('../input/previous_application.csv')
#installments_payments = pd.read_csv('../input/installments_payments.csv')
#credit_card_balance = pd.read_csv('../input/credit_card_balance.csv')
#bureau = pd.read_csv('../input/bureau.csv')
#application_test = pd.read_csv('../input/application_test.csv')

## application_train

In [3]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## POS_CASH_balance data

In [4]:
POS_CASH_balance.head()

NameError: name 'POS_CASH_balance' is not defined

## bureau_balance data

In [ ]:
bureau_balance.head()

## previous_application data

In [ ]:
previous_application.head()

## installments_payments data

In [ ]:
installments_payments.head()

## credit_card_balance data

In [ ]:
credit_card_balance.head()

## bureau data

In [ ]:
bureau.head()

# drop id, target

In [ ]:

train_data = application_train.drop(['TARGET', 'SK_ID_CURR'], axis=1)
train_data.head()

## train_missing

In [ ]:
#total = application_train.isnull().sum()#.sort_values(ascending = False)
train_missing = []
for var in train_data.columns:
    if train_data[var].isnull().sum() != 0:
        train_missing.append(var)
print(train_missing)

## application_discrete_features

In [ ]:
cnt = 0

train_discrete_features = {}
for var in train_data.columns:
    #application_discrete_features[var] = 1.*application_train[var].nunique()/application_train[var].count() < 0.001 #or some other threshold
    train_discrete_features[var] = 1.*train_data[var].nunique() < 10

discrete_features = []
for f in train_data.columns:
    if train_discrete_features[f] == True:
        discrete_features.append(f)
        
print(discrete_features)
print(len(discrete_features))

## discrete to onehot

In [ ]:

x_descrete = pd.DataFrame()

for f in train_data.columns:
    if train_discrete_features[f] == True:
        print(f)
        onehot = pd.get_dummies(train_data[f])
        onehot.columns = [f + '_' + str(col) for col in onehot.columns]
        x_descrete = pd.concat([x_descrete, onehot],axis=1)

print(x_descrete.shape)
x_descrete.head()


## continuous features
### replace NA with mean, normalize

In [ ]:
from sklearn import preprocessing

train_number_features = train_data.select_dtypes(include='number').columns.values

x_continuous = pd.DataFrame()

continuous_features = []

for f in train_number_features:
    if train_discrete_features[f] == False:
        #print(f)
        
        no_na = train_data[f].fillna(train_data[f].mean())
        no_na = no_na.values.astype(float).reshape(-1, 1)

        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(no_na)
        df = pd.DataFrame(x_scaled, columns=[f])
        
        x_continuous = pd.concat([x_continuous, df],axis=1)
        
        continuous_features.append(f)
        cnt += 1

#x_continuous.head()

print(continuous_features)

## combine X, y

In [ ]:
from sklearn.model_selection import train_test_split

X = pd.concat([x_continuous, x_descrete],axis=1)
y = application_train.TARGET
#X.head()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(x_continuous.shape)
print(x_descrete.shape)
print(X.shape)

discrete_features_all = x_descrete.columns

## PCA

In [ ]:

from sklearn.decomposition import PCA

pca = PCA(n_components = 10)

principalComponents = pca.fit_transform(X_train)

principalDf = pd.DataFrame(data = principalComponents)

Xpca = principalDf

principalDf.head()


## Train

In [ ]:
#---------------------------------------SVM -------------------------------------------------
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

X_train = Xpca

# Batch
def iter_minibatches(chunksize):
    
    # Provide chunks one by one
    chunkstartmarker = 0
    numtrainingpoints = X_train.shape[0]
    
    while chunkstartmarker < numtrainingpoints:
        #chunkrows = range(chunkstartmarker,chunkstartmarker+chunksize)
        #X_chunk, y_chunk = getrows(chunkrows)
        start = chunkstartmarker
        end = chunkstartmarker+chunksize
        X_chunk = X_train.iloc[start:end]
        y_chunk = y_train.iloc[start:end]
        yield X_chunk, y_chunk, chunkstartmarker
        chunkstartmarker += chunksize
        
def train():
    batcherator = iter_minibatches(chunksize=10000)
    model = SGDClassifier()
 
    # Train model
    for X_chunk, y_chunk, chunkstartmarker in batcherator:
        model.partial_fit(X_chunk, y_chunk, classes=np.unique(y_chunk))
        print(chunkstartmarker)
        #print(model.score(X_test, y_test))
        #print(model.predict(X_test))
    
    #accu = model.score(X_test, y_test)
    accu = model.score(pca.transform(X_test), y_test)#pca
    print('accu = ', accu)
 
    # Now make predictions with trained model
    #y_predicted = model.predict(X_test)

    joblib.dump(model, 'save/sgd.pkl')
    
train()

y.head()

In [ ]:
'''
from sklearn.svm import SVC

classifier =  SVC()

classifier.fit(X, y)
print(classifier.score(X_test, y_test))
'''

In [ ]:
'''
from sklearn.externals import joblib

joblib.dump(model, 'save/clf.pkl')

model = joblib.load('save/clf.pkl')

print(classifier.predict(X[0:1]))
'''

## output result

In [ ]:
from sklearn.externals import joblib
#discrete_features
#continuous_features

def output():
    application_test = pd.read_csv('../input/application_test.csv')
    classifier = joblib.load('save/sgd.pkl')
    
    # descrete
    
    test_descrete = pd.DataFrame()
    for f in discrete_features:
        onehot = pd.get_dummies(application_test[f])# this will loss some features with 0s
        onehot.columns = [f + '_' + str(col) for col in onehot.columns]
        test_descrete = pd.concat([test_descrete, onehot],axis=1)
    
    for f in discrete_features_all:# fill missing descrete features
        if f not in test_descrete.columns:
            test_descrete[f]= 0
    
    for f in test_descrete.columns:# remove features not in training data
        if f not in discrete_features_all:
            test_descrete = test_descrete.drop(columns=[f])
    
    # continuous
    
    test_continuous = pd.DataFrame()
    for f in continuous_features:
        no_na = application_test[f].fillna(train_data[f].mean())
        no_na = no_na.values.astype(float).reshape(-1, 1)

        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(no_na)
        df = pd.DataFrame(x_scaled, columns=[f])
        
        test_continuous = pd.concat([test_continuous, df],axis=1)
    

    test_data = pd.concat([test_continuous, test_descrete],axis=1)
    
    #print(test_continuous.shape)
    #print(test_descrete.shape)
    print(test_data.shape)
    #print(len(discrete_features))
    #print(test_descrete.columns)
    #print(x_descrete.columns)
    #print(len(discrete_features_all))
    
    #p = classifier.predict(test_data)
    p = classifier.predict(pca.transform(test_data))#pca
    #print(pd.DataFrame(data = pca.transform(test_data)).head())
    print(p)
    
    result = pd.DataFrame()
    result['SK_ID_CURR'] = application_test['SK_ID_CURR']
    result['TARGET'] = p
    print(result.head())
    
    result.to_csv('./output/sgd.csv', index = False)
    
output()